In [ ]:
import openai
from PIL import Image
import requests

In [2]:
client = openai.Client()

### Gerar Imagens

In [7]:
nome="robo_praia"
model="dall-e-3"
#prompt="Crie uma imagem com dois robôs em uma praia lendo livro de finanças"
prompt="Crie uma imagem com um homem e uma mulher velejando no mar com ondas suaves e ilhas ao redor e o veleiro levemente inclinado"
size="1024x1024"
quality="hd"
style="vivid"

In [8]:
resposta = client.images.generate(
    model=model,
    prompt=prompt,
    size=size,
    quality=quality,
    style=style,
    n=1)

In [9]:
resposta.data[0].revised_prompt

'Create an image featuring a Middle-Eastern man and a Hispanic woman sailing on an ocean with gentle waves. Surround them with islands in the distance. The sailboat should have a slight tilt to indicate movement and breeze. The scenic view should hint towards a tranquil and refreshing atmosphere with clear skies overhead.'

In [10]:
nome_arquivo = f"files/{nome}_{model}_{quality}_{style}.jpg"
image_url = resposta.data[0].url
image_data = requests.get(image_url).content
with open(nome_arquivo, "wb") as f:
    f.write(image_data)

### Image to Text

In [11]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "Descreva a imagem fornecida"},
            {"type": "image_url", "image_url": {
                "url": "https://harus.ind.br/wp-content/uploads/2023/02/5-praias-brasileiras.jpg"
            }}
        ]
    }]
)

In [13]:
resposta.choices[0].message.content

'A imagem retrata uma praia serena durante o pôr do sol. Há palmeiras inclinadas em direção à água, com a areia clara à frente. O céu está tingido de tons quentes, como laranja e rosa, refletindo-se nas águas tranquilas do mar. O ambiente é calmo, com algumas sombras de árvores e estruturas ao fundo, sugerindo um lugar tropical e relaxante. A cena evoca uma atmosfera de tranquilidade e beleza natural.'

### Extraia placa de carro

In [14]:
texto = "Qual é a placa? Devolva apenas a placa!"

In [15]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": texto},
            {"type": "image_url", "image_url": {
                "url": "https://quatrorodas.abril.com.br/wp-content/uploads/2018/09/imagem-e1536873662306.jpg?quality=70&strip=info&w=720&crop=1"
            }}
        ]
    }],
    max_tokens=1000
)

In [16]:
resposta.choices[0].message.content

'R102A19'

### OCR

In [17]:
texto = "O que está escrito na imagem?"

In [18]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": texto},
            {"type": "image_url", "image_url": {
                "url": "https://www.rbsdirect.com.br/imagesrc/17752154.jpg?w=700"
            }}
        ]
    }],
    max_tokens=1000
)

In [19]:
resposta.choices[0].message.content

'Na imagem, está escrito:\n\n"ISSO É UM PEDIDO:\n\nQue você seja muito feliz no seu grande dia.\nEste vestido tem mãos de uma brasileira, gera emprego e movimenta a economia local.\n\nFoi um prazer fazê-lo para você."\n\nAssinado: Elianna, Bastidores, 1/3.'

### Comparação de 2 imagens para identificar erros

In [20]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What are in these images? Is there any difference between them?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

In [21]:
response.choices[0].message.content

'The images depict a serene landscape featuring a wooden walkway that traverses a lush, green field under a blue sky with clouds. The scenes appear quite similar, with both showcasing the same general setting, composition, and natural elements.\n\nIf there are differences, they might be subtle and could include variations in color saturation, brightness, or minor details in the landscape. However, without more specific context about what to look for, the images seem largely identical in their overall representation of the landscape.'